In [2]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os
from google import genai
import json
from pydantic import BaseModel
import time
import pymupdf
import tempfile

In [3]:
anexos = pd.read_excel('docs/jusbrasil.xlsx', sheet_name="Anexos")

In [4]:
anexos_copia = anexos.copy().loc[:,["processoID", "processoAnexoID", "Download copia", "Tipo de anexo", "Publicado em"]]
sentencas = anexos_copia.loc[anexos["Tipo de anexo"].isin(["SENTENCA"]),
                                           ["processoID", "processoAnexoID", "Download copia", "Publicado em"]]
sentencas.head()

,processoID,processoAnexoID,Download copia,Publicado em
1,474474022,2344968081,http://jud-anexos.digesto.com.br/0cacd6d80c499...,2021-08-13
7,474474806,2294638461,http://jud-anexos.digesto.com.br/90df1f9ac9917...,2022-05-16
33,575718248,1511375012,http://jud-anexos.digesto.com.br/c89c7a26febcc...,2022-07-20
87,580649179,1504080402,http://jud-anexos.digesto.com.br/585fdfebfdd6d...,2024-07-31
105,654928926,2419043700,http://jud-anexos.digesto.com.br/6be7e9beac86d...,2025-01-29


In [5]:
def identifica_acesso_negado(processos):
    copia_processos = processos.copy()
    acessos_negados = []

    for i in range(copia_processos.shape[0]):
        link = copia_processos.iloc[i, 2]
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        texto = ""

        if 'html' in content_type:
            # print(f"[HTML] Extraindo de: {link}")
            soup = BeautifulSoup(response.content, 'html.parser')
            texto = soup.get_text()

        match = re.search(r'Acesso negado', texto)
        if match:
            acessos_negados.append(i)
            continue
    
    return copia_processos.iloc[acessos_negados, :]

df_html_only = sentencas[sentencas['Download copia'].str.contains(r'\.html?$', case=False, na=False)]
acessos_negados = identifica_acesso_negado(df_html_only)
acessos_negados.head()

,processoID,processoAnexoID,Download copia,Publicado em
1293,471515190,727395468,http://jud-anexos.digesto.com.br/3a9fbbe2c4f69...,2022-04-28
1302,471515190,727395467,http://jud-anexos.digesto.com.br/7767cbbc02e35...,2024-03-07
2876,595998611,1403409515,http://jud-anexos.digesto.com.br/0ec3e0c015d72...,2024-08-05
2877,595998611,1403409516,http://jud-anexos.digesto.com.br/385306277d658...,2024-08-05
8000,690025576,2015456650,http://jud-anexos.digesto.com.br/fb01c9fd839aa...,2021-01-01


In [ ]:
# Dropando os acessos negados
# sentencas_acessaveis = sentencas[sentencas["processoAnexoID"].isin(acessos_negados["processoAnexoID"]) == False]

# Exportando sentenças sem acesso negado para um arquivo Excel
# sentencas_acessaveis.to_excel("docs/sentencas_acessaveis.xlsx", index=False)
sentencas_acessaveis = pd.read_excel("docs/sentencas_acessaveis.xlsx")

In [7]:
# Verificando quantidade de processos após o drop
sentencas_acessaveis["processoID"].unique().shape

(223,)

In [8]:
ids_processos = sentencas_acessaveis["processoID"].unique()
processos_com_mais_de_uma_sentenca = []

for id in ids_processos:
    linhas_correspondentes = sentencas_acessaveis.loc[sentencas['processoID'] == id]
    if linhas_correspondentes.shape[0] > 1:
        processos_com_mais_de_uma_sentenca.append(int(id))

print(f"Número de processos com mais de uma sentença: {len(processos_com_mais_de_uma_sentenca)}")

Número de processos com mais de uma sentença: 84


In [13]:
sentencas_final = sentencas_acessaveis.copy()
for id in ids_processos:
    linhas_correspondentes = sentencas_acessaveis.loc[sentencas_acessaveis['processoID'] == id]
    if linhas_correspondentes.shape[0] > 1:
        if linhas_correspondentes['Publicado em'].nunique() == 1:
            # Data de publicação igual: eliminando por ordem de anexo id
            linhas_correspondentes = linhas_correspondentes.sort_values(by=["processoAnexoID"], ascending=True)
            manter_id = linhas_correspondentes.iloc[0]['processoAnexoID']
        else:
            # Ordenando as linhas por data
            linhas_correspondentes = linhas_correspondentes.sort_values(by=["Publicado em"], ascending=True)
            # Mantendo a primeira linha (mais antiga)
            manter_id = linhas_correspondentes.iloc[0]['processoAnexoID']
        
        # Eliminar todas as outras com mesmo processoID e processoAnexoID diferente do que foi mantido
        sentencas_final = sentencas_final[~((sentencas_final["processoID"] == id) & (sentencas_final["processoAnexoID"] != manter_id))]

In [18]:
# Verificando dataframe após a eliminação
ids_processos = sentencas_final["processoID"].unique()
processos_com_mais_de_uma_sentenca = []

for id in ids_processos:
    linhas_correspondentes = sentencas_final.loc[sentencas['processoID'] == id]
    if linhas_correspondentes.shape[0] > 1:
        processos_com_mais_de_uma_sentenca.append(int(id))

print(f"Número de processos com mais de uma sentença: {len(processos_com_mais_de_uma_sentenca)}")

Número de processos com mais de uma sentença: 0


In [16]:
# Verificando se a quantidade de processos após o drop se manteve
sentencas_final["processoID"].unique().shape

(223,)

In [ ]:
# Exportando df final de sentenças para um arquivo Excel
# sentencas_final.to_excel("docs/sentencas_final.xlsx", index=False)
sentencas_final = pd.read_excel("docs/sentencas_final.xlsx")

In [29]:
def verifica_dano_ambiental(texto):
    """
    Função que verifica se o texto possui dano ambiental.
    Se houver, justifica a resposta. Caso contrário, responde "Não há dano ambiental".
    """
    
    prompt = f"""
    Você é um especialista em direito ambiental. 
    Você deve analisar o seguinte texto e verificar se há dano ambiental.
    Se houver, justifique sua resposta em até 20 palavras. Caso contrário, responda "Não há dano ambiental".
    Refira-se EXCLUSIVAMENTE ao texto fornecido para identificar se há dano ambiental e para justificar sua resposta!!

    Texto: {texto}
    """
    
    class FormatoResposta(BaseModel):
        isDanoAmbiental: bool
        justificativa: str

    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    resposta = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            'response_schema': FormatoResposta,
            'temperature': 1.0
            # 'max_output_tokens': 500,
        }
    )
    time.sleep(3)
    return resposta

In [30]:
respostas = []

for id in ids_processos:
    linha = sentencas_final.loc[sentencas_final['processoID'] == id]
    texto = ''
    linha = linha.iloc[0] # extrai a linha como Series

    processoAnexoID = linha['processoAnexoID']
    link = linha['Download copia']
    response = requests.get(link)
    response.encoding = 'utf-8'
    content_type = response.headers.get('Content-Type', '')
    
    if 'pdf' in content_type:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(response.content)
            tmp_path = tmp_file.name

        with pymupdf.open(tmp_path) as doc:
            for page in doc:
                texto += page.get_text()
    
    elif 'html' in content_type:
        soup = BeautifulSoup(response.content, 'html.parser')
        texto += soup.get_text()

    resposta_prompt = verifica_dano_ambiental(texto)
    resposta_prompt = json.loads(resposta_prompt.text)
    resposta_prompt["processoAnexoID"] = processoAnexoID
    resposta_prompt["link_referencia"] = link
    resposta_prompt["processoID"] = id
    respostas.append(resposta_prompt)

respostas

[{'isDanoAmbiental': True,
  'justificativa': 'A empresa possuía madeira em tora sem licença válida, conforme Auto de Infração, configurando dano ambiental.',
  'processoAnexoID': np.int64(2344968081),
  'link_referencia': 'http://jud-anexos.digesto.com.br/0cacd6d80c499ae25dcb85380a07c3dd.pdf',
  'processoID': np.int64(474474022)},
 {'isDanoAmbiental': True,
  'justificativa': 'Sim, houve destruição de 121,15 hectares de floresta nativa no bioma amazônico sem autorização ambiental.',
  'processoAnexoID': np.int64(2294638461),
  'link_referencia': 'http://jud-anexos.digesto.com.br/90df1f9ac9917f6df4b9f91915b3a8bd.pdf',
  'processoID': np.int64(474474806)},
 {'isDanoAmbiental': False,
  'justificativa': 'Não há dano ambiental, pois o texto trata de uma ação de venda casada de carregadores de celular.',
  'processoAnexoID': np.int64(1511375012),
  'link_referencia': 'http://jud-anexos.digesto.com.br/c89c7a26febccd44703bb340de170f09.pdf',
  'processoID': np.int64(575718248)},
 {'isDanoAmbi

In [31]:
# Transformando a lista de respostas em um DataFrame
respostas_df = pd.DataFrame(respostas)

# Reorganizando as colunas
respostas_df = respostas_df[["processoID", "processoAnexoID", "isDanoAmbiental", "justificativa", "link_referencia"]]

respostas_df.head()

,processoID,processoAnexoID,isDanoAmbiental,justificativa,link_referencia
0,474474022,2344968081,True,A empresa possuía madeira em tora sem licença ...,http://jud-anexos.digesto.com.br/0cacd6d80c499...
1,474474806,2294638461,True,"Sim, houve destruição de 121,15 hectares de fl...",http://jud-anexos.digesto.com.br/90df1f9ac9917...
2,575718248,1511375012,False,"Não há dano ambiental, pois o texto trata de u...",http://jud-anexos.digesto.com.br/c89c7a26febcc...
3,580649179,1504080402,False,"Não há dano ambiental, pois o texto trata de i...",http://jud-anexos.digesto.com.br/585fdfebfdd6d...
4,654928926,2419043700,False,Não há dano ambiental. O texto trata de cobran...,http://jud-anexos.digesto.com.br/6be7e9beac86d...


In [3]:
# Exportando df de classificação de sentenças para um arquivo Excel
# respostas_df.to_excel("docs/respostas_classificacao_sentencas.xlsx", index=False)
respostas_df = pd.read_excel("docs/respostas_classificacao_sentencas.xlsx")

In [4]:
sentencas_danos_ambientais = respostas_df.loc[respostas_df["isDanoAmbiental"] == True]

In [5]:
lista_processos = list(sentencas_danos_ambientais['processoID'].unique())

# Dividindo a lista de processos em 4 partes menores iguais
def divide_lista_em_partes(lista, num_partes):
    if len(lista) % num_partes == 0:
        # Dvisão exata
        tamanho_parte = len(lista) // num_partes
        partes = [lista[i:i + tamanho_parte] for i in range(0, len(lista), tamanho_parte)]
    else:
        # Coloca o resto na última parte
        tamanho_parte = len(lista) // num_partes
        partes = []
        for i in range(0, num_partes):
            if i == num_partes - 1:
                partes.append(lista[i * tamanho_parte:])
            else:
                partes.append(lista[i * tamanho_parte:(i + 1) * tamanho_parte])
    return partes

partes = divide_lista_em_partes(lista_processos, 4)
print("Distribuição de processos entre as partes:")
for i, parte in enumerate(partes):
    print(f"Parte {i + 1}: {len(parte)} processos")
print("Total de processos:", len(lista_processos))

Distribuição de processos entre as partes:
Parte 1: 22 processos
Parte 2: 22 processos
Parte 3: 22 processos
Parte 4: 24 processos
Total de processos: 90


In [ ]:
def analisa_sentenca(texto_extraido):
    """
    Função que extrai informações de um texto judicial relacionado a danos ambientais.
    O texto deve ser um texto bruto que descreve um processo judicial relacionado a algum dano ambiental.
    """
    
    prompt = f"""
        SYSTEM: Você é meu assistente especialista em análise e extração de elementos de textos judiciais. Você irá realizar extrações especificamente sobre
        danos socioambientais de diversos tipos, pensando em futuramente usar a tabela gerada para fazer uma modelagem preditiva de multas para danos socioambientais.

        INSTRUCTIONS:
        Você receberá um texto bruto que descreve um processo judicial relacionado a algum dano ambiental.
        Seu objetivo é analisar esse texto e extrair 15 informações específicas para montar um banco de dados, conforme as instruções abaixo.
        Definição útil: Dano ambiental é qualquer prejuízo causado ao meio ambiente (água, solo, ar, fauna, flora, patrimônio paisagístico etc.) por ação ou omissão
        de um terceiro, sendo gerador de obrigação de reparação, conforme os artigos 186 e 927 do Código Civil.
        Retorno esperado:

        [0] Número do processo judicial no formato "0000000-00.0000.0.00.0000"
         Se não houver, retorne: NULL
        
        [1] Georreferência do local afetado, no formato: XX°xx’xx.xx” S e XX°xx’xx.xx” O
         Se não houver, retorne: NULL

        [2] Sigla da Unidade Federativa (UF) (ex: "SP", "MG" etc.)
         Se não houver, retorne: NULL

        [3] Município ou cidade do local afetado. (ex: "São Paulo", "Minas Gerais" etc)
         Se não houver, retorne: NULL

        [4] Nome do responsável pelo dano ambiental (empresa ou pessoa física).
         Se não houver, retorne: NULL

        [5] Categoria do responsável: "Pessoa Física" ou "Pessoa Jurídica"
         Se não houver, retorne: NULL

        [6] Tipo de impacto: Categoria do dano (ex.: "Desmatamento de APP", "Derramamento de Petróleo", "Poluição Hídrica").
        Se não houver, retorne: NULL

        [7] Descrição do impacto: Resuma o impacto em até 30 palavras.
        Se não houver, retorne: NULL

        [8] Data do impacto ambiental no formato: DD/MM/AA
         Se não houver, retorne: NULL

        [9] Extensão da área afetada (ex: "15000".)
         Se não houver, retorne: NULL
        Sempre em metros quadrados (m²) ou hectares (ha), então converta se necessário. Não use separadores para milhares

        [10] Unidade de medida da área (ex: "ha" ou "m2")
         Se não houver, retorne: NULL

        [11] Houve compensação não monetária atribuída (ou seja, alguma ação de reparação ambiental, como reflorestamento, recuperação de áreas degradadas, etc.)
         Se houver, retorne: "True"
         Se não houver, retorne: "False"

        [12] Categoria da compensação: "Multas Administrativas", "Compensações Financeiras", "Obrigações de Fazer (com custo)", "Custas Judiciais e Acordos" ou "Valoração Econômica".
         Se não houver, retorne: NULL
         Tente não fugir dessas categorias.

        [13] Tipo de multa: (0, 1 ou 2)
        Se for uma única multa aplicada sobre o responsável, coloque 0
        Se for algo como uma multa diária, coloque 1.
        Se forem os dois, como uma multa imediata e um pagamento diário, coloque 2

        [14] Valor da multa completa para o Tipo 0 ou 2: (ex: "123000.00", "999999.99")
        Caso o item 12 seja 0, complete essa coluna com o valor total da multa aplicada.
        Caso o item 12 seja 1, deixe como NULL
        Caso o item 12 seja 2, complete essa coluna com o valor total da multa imediata aplicada.

        [15] Valor da multa diária para o Tipo 1 ou 2: (ex: "123000.00", "10000.50")
        Caso o item 12 seja 1 ou 2, complete essa coluna com o valor diário a ser pago pelo responsável.
        Caso o item 12 seja 0, deixe como NULL

        IMPORTANTE: A string será usada para um dataframe posteriormente. Logo:
        NÃO inclua explicações, formatações extras, aspas ou colchetes na resposta.
        NÃO deixe nenhuma casa em branco ou vazia. SEMPRE coloque NULL caso não encontre no texto.
        NÃO inclua o texto original, nem explicações, apenas a string limpa.
        A ordem dos elementos PRECISA SER RESPEITADA!! NENHUM ELEMENTO PODE VIR ANTES DA SUA POSIÇÃO!!

        USER:
        Texto para análise:
        {texto_extraido}
    """
    
    class FormatoResposta2(BaseModel):
        numero_processo: str
        georreferencia: str
        uf: str
        municipio: str
        responsavel: str
        categoria_responsavel: str
        tipo_impacto: str
        descricao_impacto: str
        data_impacto: str
        area_afetada: str
        unidade_area: str
        houve_compensacao: bool
        categoria_compensacao: str
        tipo_multa: int | str
        valor_multa: float | str
        valor_multa_diaria: float | str

    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    resposta = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            'response_schema': FormatoResposta2,
            'temperature': 1.0
            # 'max_output_tokens': 500,
        }
    )
    time.sleep(2)
    return resposta

In [7]:
# Analisando os processos com dano ambiental
respostas_danos_ambientais = []
for id in partes[0]:
    linha = sentencas_danos_ambientais.loc[sentencas_danos_ambientais['processoID'] == id]
    texto = ''
    linha = linha.iloc[0] # extrai a linha como Series

    processoAnexoID = linha['processoAnexoID']
    link = linha['link_referencia']
    response = requests.get(link)
    response.encoding = 'utf-8'
    content_type = response.headers.get('Content-Type', '')
    
    if 'pdf' in content_type:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(response.content)
            tmp_path = tmp_file.name

        with pymupdf.open(tmp_path) as doc:
            for page in doc:
                texto += page.get_text()
    
    elif 'html' in content_type:
        soup = BeautifulSoup(response.content, 'html.parser')
        texto += soup.get_text()

    resposta_prompt = analisa_sentenca(texto)
    resposta_prompt = json.loads(resposta_prompt.text)
    resposta_prompt["processoAnexoID"] = processoAnexoID
    resposta_prompt["link_referencia"] = link
    resposta_prompt["processoID"] = id
    respostas_danos_ambientais.append(resposta_prompt)

respostas_danos_ambientais

[{'numero_processo': '0600357-89.2021.8.04.5600',
  'georreferencia': 'NULL',
  'uf': 'AM',
  'municipio': 'Manicoré',
  'responsavel': 'DESTAQUE INDUSTRIA E COMERCIO DE MADEIRAS LTDA',
  'categoria_responsavel': 'Pessoa Jurídica',
  'tipo_impacto': 'Depósito irregular de madeira',
  'descricao_impacto': 'A empresa mantinha em depósito 288,03 metros cúbicos de madeira em tora sem licença válida.',
  'data_impacto': '30/04/2016',
  'area_afetada': 'NULL',
  'unidade_area': 'NULL',
  'houve_compensacao': True,
  'categoria_compensacao': 'Custas Judiciais e Acordos',
  'tipo_multa': '2',
  'valor_multa': 'NULL',
  'valor_multa_diaria': 'NULL',
  'processoAnexoID': np.int64(2344968081),
  'link_referencia': 'http://jud-anexos.digesto.com.br/0cacd6d80c499ae25dcb85380a07c3dd.pdf',
  'processoID': np.int64(474474022)},
 {'numero_processo': '0600384-72.2021.8.04.5600',
  'georreferencia': '07°43’11.00” S e 61°28’11.00” O',
  'uf': 'AM',
  'municipio': 'Manicoré',
  'responsavel': 'CARLOS ALEXA

In [9]:
# Transformando a lista de respostas de danos ambientais da primeira parte em um DataFrame
respostas_danos_ambientais_df_parte0 = pd.DataFrame(respostas_danos_ambientais)

# Reorganizando as colunas
respostas_danos_ambientais_df_parte0 = respostas_danos_ambientais_df_parte0[["numero_processo", "processoID", "processoAnexoID", "georreferencia", "uf", "municipio", "responsavel", "categoria_responsavel", "tipo_impacto", "descricao_impacto", "data_impacto", "area_afetada", "unidade_area", "houve_compensacao", "categoria_compensacao", "tipo_multa", "valor_multa", "valor_multa_diaria", "link_referencia"]]

respostas_danos_ambientais_df_parte0

,numero_processo,processoID,processoAnexoID,georreferencia,uf,municipio,responsavel,categoria_responsavel,tipo_impacto,descricao_impacto,data_impacto,area_afetada,unidade_area,houve_compensacao,categoria_compensacao,tipo_multa,valor_multa,valor_multa_diaria,link_referencia
0,0600357-89.2021.8.04.5600,474474022,2344968081,NULL,AM,Manicoré,DESTAQUE INDUSTRIA E COMERCIO DE MADEIRAS LTDA,Pessoa Jurídica,Depósito irregular de madeira,"A empresa mantinha em depósito 288,03 metros c...",30/04/2016,NULL,NULL,True,Custas Judiciais e Acordos,2,NULL,NULL,http://jud-anexos.digesto.com.br/0cacd6d80c499...
1,0600384-72.2021.8.04.5600,474474806,2294638461,07°43’11.00” S e 61°28’11.00” O,AM,Manicoré,CARLOS ALEXANDRE ROSSI,Pessoa Física,Desmatamento de vegetação nativa,"Destruição de 121,15 hectares de floresta nati...",16/06/2016,1211500,m2,True,Obrigações de Fazer (com custo),0,NULL,NULL,http://jud-anexos.digesto.com.br/90df1f9ac9917...
2,0600592-22.2022.8.04.5600,682231747,1915313002,NULL,AM,Manicoré,C R MADEIREIRA AMAZONAS LTDA-EPP,Pessoa Jurídica,Queima de Rejeitos Industriais,Lançamento e queima de rejeitos de indústria m...,07/11/2018,NULL,NULL,True,Obrigações de Fazer (com custo),2,NULL,10000.00,http://jud-anexos.digesto.com.br/209b98634c9c4...
3,1000009-75.2018.4.01.3903,311524686,269166848,NULL,PA,Altamira,MARCIO SILVA VALE,Pessoa Física,Desmatamento,Ação civil pública movida por desmatamento e i...,NULL,NULL,NULL,False,NULL,0,NULL,NULL,http://jud-anexos.digesto.com.br/7cef4bd654c35...
4,1000305-06.2018.4.01.3901,326977847,1526470307,NULL,PA,Marabá,VALE S.A.,Pessoa Jurídica,Desmatamento de APP,"Supressão de castanheiras, poluição hídrica do...",NULL,54000,m2,True,Obrigações de Fazer (com custo),0,NULL,NULL,http://jud-anexos.digesto.com.br/d5d876cd4f1b1...
5,1003105-10.2017.4.01.3200,486374400,113620662,NULL,AM,Manicoré,Aparecida Fermino Marques,Pessoa Física,Desmatamento de Floresta Amazônica,"Desmatamento de 133,11 hectares de Floresta Am...",NULL,1331100,m2,True,Obrigações de Fazer (com custo),2,1423100.16,100000.00,http://jud-anexos.digesto.com.br/24586eb78e174...
6,5096053-69.2018.8.09.0011,497378463,2868300725,NULL,GO,Aparecida de Goiânia,W. S. Lavanderia LTDA-ME,Pessoa Jurídica,Poluição Hídrica,"Lançamento de efluentes (tinta, produtos quími...",NULL,NULL,NULL,False,Custas Judiciais e Acordos,0,15000.00,NULL,http://jud-anexos.digesto.com.br/be70db0dae0a1...
7,5842003-89.2023.8.09.0074,741569033,2451722545,NULL,GO,Ipameri,EQUATORIAL GOIÁS DISTRIBUIDORA DE ENERGIA S.A,Pessoa Jurídica,Falha no fornecimento de energia elétrica,Falhas no fornecimento de energia elétrica que...,NULL,NULL,NULL,True,Custas Judiciais e Acordos,0,38285.91,NULL,http://jud-anexos.digesto.com.br/5631abc57e085...
8,0050002-26.2020.8.06.0035,435827204,2254516518,NULL,CE,Aracati,Oscar Noce,Pessoa Física,Ocupação Irregular de Terreno de Marinha,Ocupação irregular de barraca de praia em terr...,09/01/2020,NULL,m2,True,Obrigações de Fazer (com custo),0,NULL,NULL,http://jud-anexos.digesto.com.br/a322a1e63179a...
9,0002171-93.2018.8.08.0035,654920222,2583016248,NULL,ES,Vila Velha,A IGREJA,Pessoa Jurídica,Poluição Sonora,Emissão de som em violação aos limites de horá...,NULL,NULL,NULL,False,NULL,0,NULL,NULL,http://jud-anexos.digesto.com.br/f8941539197bd...


In [ ]:
# Exportando df de classificação de sentenças para um arquivo Excel
# respostas_danos_ambientais_df_parte0.to_excel("docs/respostas_danos_ambientais_df_parte0.xlsx", index=False)